In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install jsonlines
!pip install pymorphy2
!pip install simpletransformers

In [ ]:
import pandas as pd
import numpy as np
import json
import os, re
import random
from sklearn.metrics import matthews_corrcoef
import jsonlines
import time
from scipy.special import softmax
 
from simpletransformers.classification import (
    ClassificationModel, ClassificationArgs
)
 
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [ ]:
file = "/content/drive/MyDrive/ColabFiles/AIIJC/newTrain.jsonl"

train = []
for line in open(file, 'r',encoding='utf-8'):
    train.append(json.loads(line))

trainData = pd.DataFrame(train)
trainData

,premise,hypothesis,label,idx,language,premiseTrans,hypothesisTrans
0,Vid ett möte i regionfullmäktige av guvernören...,Tjänstemän är nöjda med resultatet av året.,entailment,0,sv,At a meeting of the Regional Assembly of the G...,Officials are satisfied with the result of the...
1,Und dann hat er seine Güte nicht mitgezählt. D...,Der Dieb brach in ein unbedecktes Auto ein.,entailment,1,de,And then he didn&#39;t count his kindness. The...,The thief broke into an uncovered car.
2,Men på grund av den svåra ekonomiska situation...,Samlare slutade kalla honom.,not_entailment,2,sv,"However, due to the difficult financial situat...",Collectors stopped calling him.
3,"Рассказал, что знаком с обвиняемым около 5 лет...",Свидетель ходил вместе с обвиняемым на футбол.,not_entailment,3,ru,He said that he had known the accused for abou...,The witness went to football with the accused.
4,"Chaque jour, les correspondants de service par...",Il y a de nombreux correspondants dans la ville.,not_entailment,4,fr,"Every day, the service correspondents will tal...",There are many correspondents in the city.
...,...,...,...,...,...,...,...
9379,Les sauveteurs ont déclaré que les points sont...,Les victimes recevront de l'aide.,entailment,9379,fr,"Rescuers said the points are always ready and,...",Victims will receive help.
9380,"Themen, über die die Delegierten heute aktiv d...",Die Delegierten aus dem Gebiet Penza diskutier...,not_entailment,9380,de,Topics that the delegates are actively discuss...,The delegates from the Penza area discuss all ...
9381,Mr Gregory's conclusion in 1994 was that the c...,50th Anniversary of Normandy Landings lasts a ...,not_entailment,9381,en,Mr Gregory's conclusion in 1994 was that the c...,50th Anniversary of Normandy Landings lasts a ...
9382,Vi körde långsamt. Ett band vi missade och som...,Vi missade ett körfält.,entailment,9382,sv,We drove slowly. A band we missed and like to ...,We missed a lane.


In [ ]:
for i in range(len(trainData.label)):
  if trainData.label.iloc[i] == 'entailment': 
    trainData.label.iloc[i] = 1
  else:
    trainData.label.iloc[i] = 0

def delTrash(s):
  s = s.replace('&#39;','')
  s = s.replace('&quot;','')
  return s

trainData.premise = trainData.premise.apply(delTrash)
trainData.hypothesis = trainData.hypothesis.apply(delTrash)
trainData.premiseTrans = trainData.premiseTrans.apply(delTrash)
trainData.hypothesisTrans = trainData.hypothesisTrans.apply(delTrash)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
trainData

,premise,hypothesis,label,idx,language,premiseTrans,hypothesisTrans
0,Vid ett möte i regionfullmäktige av guvernören...,Tjänstemän är nöjda med resultatet av året.,1,0,sv,At a meeting of the Regional Assembly of the G...,Officials are satisfied with the result of the...
1,Und dann hat er seine Güte nicht mitgezählt. D...,Der Dieb brach in ein unbedecktes Auto ein.,1,1,de,And then he didnt count his kindness. The thie...,The thief broke into an uncovered car.
2,Men på grund av den svåra ekonomiska situation...,Samlare slutade kalla honom.,0,2,sv,"However, due to the difficult financial situat...",Collectors stopped calling him.
3,"Рассказал, что знаком с обвиняемым около 5 лет...",Свидетель ходил вместе с обвиняемым на футбол.,0,3,ru,He said that he had known the accused for abou...,The witness went to football with the accused.
4,"Chaque jour, les correspondants de service par...",Il y a de nombreux correspondants dans la ville.,0,4,fr,"Every day, the service correspondents will tal...",There are many correspondents in the city.
...,...,...,...,...,...,...,...
9379,Les sauveteurs ont déclaré que les points sont...,Les victimes recevront de l'aide.,1,9379,fr,"Rescuers said the points are always ready and,...",Victims will receive help.
9380,"Themen, über die die Delegierten heute aktiv d...",Die Delegierten aus dem Gebiet Penza diskutier...,0,9380,de,Topics that the delegates are actively discuss...,The delegates from the Penza area discuss all ...
9381,Mr Gregory's conclusion in 1994 was that the c...,50th Anniversary of Normandy Landings lasts a ...,0,9381,en,Mr Gregory's conclusion in 1994 was that the c...,50th Anniversary of Normandy Landings lasts a ...
9382,Vi körde långsamt. Ett band vi missade och som...,Vi missade ett körfält.,1,9382,sv,We drove slowly. A band we missed and like to ...,We missed a lane.


In [ ]:
text_a = trainData.premiseTrans.values
text_b = trainData.hypothesisTrans.values
labels = trainData.label.values

In [ ]:
result = []
for i in range(len(text_a)):
  result.append([text_a[i],text_b[i],labels[i]])

In [ ]:
train_df = pd.DataFrame(result)
train_df.columns = ["text_a","text_b","labels"]
train_df

,text_a,text_b,labels
0,At a meeting of the Regional Assembly of the G...,Officials are satisfied with the result of the...,1
1,And then he didnt count his kindness. The thie...,The thief broke into an uncovered car.,1
2,"However, due to the difficult financial situat...",Collectors stopped calling him.,0
3,He said that he had known the accused for abou...,The witness went to football with the accused.,0
4,"Every day, the service correspondents will tal...",There are many correspondents in the city.,0
...,...,...,...
9379,"Rescuers said the points are always ready and,...",Victims will receive help.,1
9380,Topics that the delegates are actively discuss...,The delegates from the Penza area discuss all ...,0
9381,Mr Gregory's conclusion in 1994 was that the c...,50th Anniversary of Normandy Landings lasts a ...,0
9382,We drove slowly. A band we missed and like to ...,We missed a lane.,1


In [ ]:
model_args = ClassificationArgs(num_train_epochs=1,overwrite_output_dir=True,train_batch_size=4,max_seq_length=128,learning_rate=4e-06,
do_lower_case=True,manual_seed=666)
#learning_rate=4e-06

In [ ]:
model = ClassificationModel("****", "****")

Downloading:   0%|          | 0.00/729 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-large-mnli were not used when initializing DebertaForSequenceClassification: ['config']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

In [ ]:
model.args = model_args

In [ ]:
model.train_model(train_df)

  0%|          | 0/9384 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/2346 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


(2346, 0.49993158817737965)

In [ ]:
test = train_df
test.pop("labels")
test

,text_a,text_b
0,At a meeting of the Regional Assembly of the G...,Officials are satisfied with the result of the...
1,And then he didnt count his kindness. The thie...,The thief broke into an uncovered car.
2,"However, due to the difficult financial situat...",Collectors stopped calling him.
3,He said that he had known the accused for abou...,The witness went to football with the accused.
4,"Every day, the service correspondents will tal...",There are many correspondents in the city.
...,...,...
9379,"Rescuers said the points are always ready and,...",Victims will receive help.
9380,Topics that the delegates are actively discuss...,The delegates from the Penza area discuss all ...
9381,Mr Gregory's conclusion in 1994 was that the c...,50th Anniversary of Normandy Landings lasts a ...
9382,We drove slowly. A band we missed and like to ...,We missed a lane.


In [ ]:
pre = []
for i in range(len(test)):
  a = [test.iloc[i]['text_a'],test.iloc[i]['text_b']]
  pre.append(a)
pre

[['At a meeting of the Regional Assembly of the Governor for the Development of Physical Culture and Sports officials said that the past year was successful for the industry.',
  'Officials are satisfied with the result of the year.'],
 ['And then he didnt count his kindness. The thiefs staff found a 42-year-old recidivist from the area. He confessed to breaking into the car because he saw the victim get out of the car and forgot to lock it.',
  'The thief broke into an uncovered car.'],
 ['However, due to the difficult financial situation, he could not get his money back. Soon he started calling collectors, they are obliged to repay the debt. Calls did not stop for a long time, and then began calling on the phone to his mother.',
  'Collectors stopped calling him.'],
 ['He said that he had known the accused for about 5 years, they went to gym together. - Adequate, normal person, but recently he did not work constantly, experienced financial difficulties. I could not get a job because 

In [ ]:
predictions, raw_outputs = model.predict(pre)

  0%|          | 0/9384 [00:00<?, ?it/s]

  0%|          | 0/1173 [00:00<?, ?it/s]

In [ ]:
predictions

array([1, 1, 0, ..., 0, 1, 1])

In [ ]:
matthews_corrcoef(predictions,train_df.labels)

0.9086286981300817

In [ ]:
file = "/content/drive/MyDrive/ColabFiles/AIIJC/newTest.jsonl"

test = []
for line in open(file, 'r',encoding='utf-8'):
    test.append(json.loads(line))

testData = pd.DataFrame(test)

testData.premise = testData.premise.apply(delTrash)
testData.hypothesis = testData.hypothesis.apply(delTrash)

testData

,idx,premise,hypothesis
0,0,Albertosaurus and other tyrannosaurids were mu...,Albertosaurus and other tyrannosaurids were mu...
1,1,Speaker of the House Paul Ryan has faced probl...,House Speaker Paul Ryan was faced with problem...
2,2,Wind passed over the meadow.,Calm winds blew across the meadow.
3,3,The tanks were developed in Great Britain and ...,Tanks were developed in Great Britain and Fran...
4,4,It reminds me of when my little brother and I ...,It reminds me of when I played Super Mario wit...
...,...,...,...
5090,5090,He was dissatisfied.,"He deceitfully proclaimed: ""This is all I ever..."
5091,5091,Lets consider some words as positive examples ...,Lets look at some contextual words as positive...
5092,5092,David Tennant is the best Doctor in the series.,"David Tennant is the best Doctor in the House,..."
5093,5093,"Just saw the first 15 minutes, tired, fled to ...","Just watched the first 10 minutes, tired, fled..."


In [ ]:
pre = []
for i in range(len(test)):
  a = [testData.iloc[i]['premise'],testData.iloc[i]['hypothesis']]
  pre.append(a)
pre

[['Albertosaurus and other tyrannosaurids were multi-toothed, with teeth of different shapes depending on their position in the mouth.',
  'Albertosaurus and other tyrannosaurids were multi-toothed.'],
 ['Speaker of the House Paul Ryan has faced problems from fellow Republicans unhappy with the style of his leadership.',
  'House Speaker Paul Ryan was faced with problems from fellow Republicans, outraged by the style of his leadership.'],
 ['Wind passed over the meadow.', 'Calm winds blew across the meadow.'],
 ['The tanks were developed in Great Britain and France and first used by the British in combat, but only with partial success.',
  'Tanks were developed in Great Britain and France and were first used in combat by the British.'],
 ['It reminds me of when my little brother and I were playing Super Mario.',
  'It reminds me of when I played Super Mario with my little brother.'],
 ['Understanding a large document requires keeping track of how entities were introduced and how they e

In [ ]:
predictions, raw_outputs = model.predict(pre)

  0%|          | 0/5095 [00:00<?, ?it/s]

  0%|          | 0/637 [00:00<?, ?it/s]

In [ ]:
counter = 0

for i in range(len(predictions)):
  if predictions[i] == 1:
    counter += 1
counter

2273

In [ ]:
with jsonlines.open("/content/drive/MyDrive/ColabFiles/AIIJC/newTest.jsonl") as reader:
    counter = 0
    answers_arr = []
 
    for obj in reader:
        ob = dict(obj)
        
        if predictions[counter] == 1:
          text_dict = {
            "idx": ob['idx'],
            "label": 'entailment'
          }
        else:
          text_dict = {
            "idx": ob['idx'],
            "label": 'not_entailment' 
          }
 
        
        counter += 1
        answers_arr.append(text_dict)
    with jsonlines.open('/content/drive/MyDrive/ColabFiles/AIIJC/submission.jsonl', mode='w') as writer:
      writer.write_all(answers_arr)